In [1]:
import curve

TRUE
[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559787


In [148]:
#from py_ecc.fields import (
#    bls12_381_FQ as FQ,
#    bls12_381_FQ2 as FQ2,
#    bls12_381_FQ12 as FQ12,
#)
from curve_field_elements import field_modulus, FQ, FQ2, FQ12
from field_helper import numberToBase, numberToArray, printEllipticPoint, Fp12convert, convert_out_to_Fp12, printFQ, printFQ2, printFQ12, print_fq12_frobenius_coeff
from miller_pseudocode import exp

In [3]:
q = curve.field_modulus
q == 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
r = curve.curve_order
x = -2**63 - 2**62 - 2**60 - 2**57 - 2**48 - 2**16
print(r)

52435875175126190479447740508185965837690552500527637822603658699938581184513


In [4]:
print(numberToArray(q, 55,7))

[35747322042231467, 36025922209447795, 1084959616957103, 7925923977987733, 16551456537884751, 23443114579904617, 1829881462546425]


In [5]:
printFQ2(curve.G2[1], 55, 7)

[
["5440959206139905",
"5515662923731907",
"10491004233861355",
"28691331721860307",
"27955817036233384",
"18631882631076677",
"907600843284315"],
["11548582681213374",
"7404909498155861",
"12768329608264924",
"12731371725497493",
"11463081752430568",
"7184576637437304",
"424072857758157"]]


In [13]:
printEllipticPoint(P=curve.G1, n=55, k=7)

[16589478066046651, 22658679592837110, 35004527604248919, 16789302793630161, 7530538873701625, 32234187716135413, 1684953952445941] [11860609209853921, 4091579406338073, 12578493816062195, 13088963032438982, 24961455755233629, 8501508834176643, 612415636564648]


In [9]:
#Fp12convert(curve.G12[1], n=55, k=7)

In [4]:
import miller_pseudocode as miller

In [5]:
from curve import double, add, neg, multiply, is_on_curve, twist, b, b2, curve_order, G1
from pairing import linefunc, cast_point_to_fq12
def miller_loop(Q, P, loop_len=abs(x), f0 = FQ12.one()):
    if Q is None or P is None:
        return FQ12.one()
    R = neg(Q)
    f = f0
    loop_bits = loop_len.bit_length()
    #print(loop_bits)
    for i in range(loop_bits - 2, -1, -1):
        f = f * f * linefunc(R, R, P)
        R = double(R)
        if loop_len & (2**i):
            f = f * f0 * linefunc(R, neg(Q), P)
            R = add(R, neg(Q))
    return f
def final_exp(f):
    return miller.exp(f, (q**12 - 1)//r)
def pairing(Q, P):
    return final_exp(miller_loop(twist(Q), cast_point_to_fq12(P)))

In [19]:
import random

privkey = random.randrange(0, r)
hashm = random.randrange(0,r)

In [20]:
pubkey = multiply(curve.G1, privkey)
Hm = multiply(curve.G2, hashm)
signature = multiply(Hm, privkey)
fakesignature = multiply(Hm, random.randrange(0,r))

In [25]:
printFQ2(Hm[0], 55, 7), printFQ2(Hm[1], 55, 7)

[
["22989519696062370",
"23807086337582295",
"25548406610603177",
"7283579713607610",
"23043507875395192",
"30305745691844326",
"1820431902321801"],
["25073846242665707",
"1491022331301032",
"14148517016494692",
"19456402486581811",
"2179787861152516",
"4544593287484104",
"1525751401169180"]]
[
["16699028771541731",
"33892025031552853",
"20275091305085540",
"7171631891112067",
"5608067197440199",
"27224833695244904",
"919565793686528"],
["20138710041507375",
"17277134432719975",
"8191063777226930",
"17756308295435455",
"1593299935390626",
"5433907724578608",
"1212723182108621"]]


(None, None)

In [9]:
pairing(Hm, pubkey) / pairing(signature, curve.G1)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [99]:
# 3-isogeny E2' to E2 coefficients
xnum = [ FQ2([0x5c759507e8e333ebb5b7a9a47d7ed8532c52d39fd3a042a88b58423c50ae15d5c2638e343d9c71c6238aaaaaaaa97d6,
                0x5c759507e8e333ebb5b7a9a47d7ed8532c52d39fd3a042a88b58423c50ae15d5c2638e343d9c71c6238aaaaaaaa97d6]),
         FQ2([0x0,
                0x11560bf17baa99bc32126fced787c88f984f87adf7ae0c7f9a208c6b4f20a4181472aaa9cb8d555526a9ffffffffc71a]),
         FQ2([0x11560bf17baa99bc32126fced787c88f984f87adf7ae0c7f9a208c6b4f20a4181472aaa9cb8d555526a9ffffffffc71e,
                0x8ab05f8bdd54cde190937e76bc3e447cc27c3d6fbd7063fcd104635a790520c0a395554e5c6aaaa9354ffffffffe38d]),
         FQ2([0x171d6541fa38ccfaed6dea691f5fb614cb14b4e7f4e810aa22d6108f142b85757098e38d0f671c7188e2aaaaaaaa5ed1,
                0x0]) ]
xden = [ FQ2([0x0,
                0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaa63]),
         FQ2( [0xc,
                0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaa9f]),
         FQ2( [0x1,
                0x0]),
         FQ2( [0,0] ) ]
ynum = [ FQ2([0x1530477c7ab4113b59a4c18b076d11930f7da5d4a07f649bf54439d87d27e500fc8c25ebf8c92f6812cfc71c71c6d706,
                0x1530477c7ab4113b59a4c18b076d11930f7da5d4a07f649bf54439d87d27e500fc8c25ebf8c92f6812cfc71c71c6d706]),
         FQ2([0x0,
                0x5c759507e8e333ebb5b7a9a47d7ed8532c52d39fd3a042a88b58423c50ae15d5c2638e343d9c71c6238aaaaaaaa97be]),
         FQ2([0x11560bf17baa99bc32126fced787c88f984f87adf7ae0c7f9a208c6b4f20a4181472aaa9cb8d555526a9ffffffffc71c,
                0x8ab05f8bdd54cde190937e76bc3e447cc27c3d6fbd7063fcd104635a790520c0a395554e5c6aaaa9354ffffffffe38f]),
         FQ2([0x124c9ad43b6cf79bfbf7043de3811ad0761b0f37a1e26286b0e977c69aa274524e79097a56dc4bd9e1b371c71c718b10,
                0x0]) ]
yden = [ FQ2([0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffa8fb,
                0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffa8fb]),
         FQ2([0x0,
                0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffa9d3]),
         FQ2([0x12,
                0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaa99]),
         FQ2([0x1,
                0x0]) ]
coeffs = [xnum, xden, ynum, yden]
coeffs_array = []
for c in coeffs:
    coeffs_array.append([[numberToArray(b.coeffs[0].n, 55, 7), numberToArray(b.coeffs[1].n, 55, 7)] for b in c])

In [105]:
#for i in range(4):
#    for j in range(4):
#        for l in range(2):
#            print(f"coeffs[{i}][{j}][{l}] = ", end="")
#            print(coeffs_array[i][j][l], end=";\n")

In [134]:
arr = [ [ numberToArray(b.n, 55, 7) for b in c.coeffs] for c in curve.G2]

In [137]:
for i in range(2):
    for j in range(2):
        print(f"g2[{i}][{j}] = ", end="")
        print(arr[i][j], end=";\n")

g2[0][0] = [95420233661880, 1773856045391785, 5044075188989616, 23082975677810779, 22805092780977487, 23121359448368568, 161253150172098];
g2[0][1] = [12522360964131710, 27624298323292107, 31369054948478259, 20553930357265165, 32921944760061193, 30416570139545860, 1398625365433447];
g2[1][0] = [5440959206139905, 5515662923731907, 10491004233861355, 28691331721860307, 27955817036233384, 18631882631076677, 907600843284315];
g2[1][1] = [11548582681213374, 7404909498155861, 12768329608264924, 12731371725497493, 11463081752430568, 7184576637437304, 424072857758157];


In [143]:
out = [ "27884767026407891",
 "4525360913426488",
 "991587327030683",
 "23335191372164909",
 "35821225693990941",
 "8502773309847038",
 "931818339881768",
 "23558617662484340",
 "1755539777993219",
 "20293744031359488",
 "19700108927825630",
 "28023258827593336",
 "7989688896250189",
 "279130932169775",
 "6999493181605224",
 "15877935146814572",
 "34605329068848078",
 "33223063651198065",
 "651941095117987",
 "15881848215121610",
 "1255304257292602",
 "29940241938544130",
 "20329595587854853",
 "1709795106623908",
 "20384090257468862",
 "1439373215787060",
 "7148972841708048",
 "220259269905836"]
nums = []
for i in range(4):
    num = 0
    for idx in range(7):
        num = num + int(out[7*i+idx]) * (2**55)**idx
    nums.append(num)

In [149]:
curve.is_on_curve( (FQ2([nums[0], nums[1]]), FQ2([nums[2],nums[3]])), curve.b2 )

True

In [152]:
curve.multiply( (FQ2([nums[0], nums[1]]), FQ2([nums[2],nums[3]])), r ) == None

True